In [1]:
from mlptools.io.read import read_from_format
from mlptools.io.write import write_from_atoms
from mlptools.utils.constants import get_all_si_atoms
from mlptools.db.db import get_session
from mlptools.db.model import Structure
import os
import pickle
import shutil
from glob import glob

In [2]:
# DBから全構造取得する
ATOMIC_SYMBOL = "Si"
PATH2ATOMS = "/Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms"

session = get_session(structure_symbol=ATOMIC_SYMBOL)
all_structure = session.query(Structure).all()

print(f"Number of dirs: {len(all_structure)}")

Number of dirs: 98396


In [3]:
ATOMIC_DISTANCE_LOWER_LIMIT = 1.0
ATOMIC_DISTANCE_UPPER_LIMIT = 6.0

all_atoms_dict = {}
for i, structure in enumerate(all_structure):
    # show progress every 1000 iter
    if i % 1000 == 0:
        print(f"{i}/{len(all_structure)}")
    
    atoms = pickle.load(open(os.path.join(PATH2ATOMS ,structure.id, "atoms.pkl"), "rb"))
    if atoms.distance_btw_nearest_neighbor is None:
        atoms.set_distance_btw_nearest_neighbor()
    
    if all_atoms_dict.get(structure.structure_name) is None:
        all_atoms_dict[structure.structure_name] = []

    if ATOMIC_DISTANCE_LOWER_LIMIT < atoms.distance_btw_nearest_neighbor["Si-Si"] < ATOMIC_DISTANCE_UPPER_LIMIT:
        all_atoms_dict[structure.structure_name].append(atoms)
    else:
        print(f"[SKIP] {structure.id}:{structure.structure_name} has {atoms.distance_btw_nearest_neighbor} A")

0/98396
[SKIP] e5970640-ed94-4db3-9052-e08959331a67:amorphous has {'Si-Si': 0.6703318355665763} A
[SKIP] 3bc017ff-341c-4b2c-b3eb-43d0a37ab18e:amorphous has {'Si-Si': 0.9350877359589113} A
[SKIP] ccd9ca96-6a31-45b4-8d33-9a7422ae10b4:amorphous has {'Si-Si': 0.9623512327493677} A
[SKIP] 4b9f1826-2299-477e-881a-382728254949:amorphous has {'Si-Si': 0.5524827922154452} A
[SKIP] 24cb5847-5f92-4621-b58c-20c058bc5787:amorphous has {'Si-Si': 0.7504380854838457} A
[SKIP] f1e38532-bd87-4377-970e-34c7521ceaea:amorphous has {'Si-Si': 0.7045600398065488} A
[SKIP] 39a82983-3482-44ca-9a5a-d2f258123b0f:amorphous has {'Si-Si': 0.4397834894740067} A
[SKIP] 796e1004-3c03-4542-ad07-6bb05cec35a1:amorphous has {'Si-Si': 0.5180524914293899} A
[SKIP] 1d92173c-9c89-47d9-a9c1-f2681414ded2:amorphous has {'Si-Si': 0.7840586792693739} A
[SKIP] 84f4370f-1710-4ec1-918d-c054e7cc3681:amorphous has {'Si-Si': 0.5883072321902075} A
[SKIP] 57511583-bf45-430d-a38f-820e83787f11:amorphous has {'Si-Si': 0.7979358080208355} A
[S

In [7]:
# 全構造の構造の種類の数を見る
for structure_name, atoms_list in all_atoms_dict.items():
    print(f"{structure_name}: {len(atoms_list)}")

amorphous: 11983
orthorhombic: 3712
diamond: 9652
trigonal: 3352
lonsdaleite: 6426
tetragonal: 5222
slab: 561
three: 100
four: 149
dimer: 80


In [5]:
# threeは多いので100個に減らす
import random

for structure_name, atoms_list in all_atoms_dict.items():
    if structure_name == "three":
        all_atoms_dict[structure_name] = random.sample(atoms_list, 100)

In [6]:
# diamondで原子数が8のものは除く
for structure_name, atoms_list in all_atoms_dict.items():
    if structure_name == "diamond":
        all_atoms_dict[structure_name] = [atoms for atoms in atoms_list if atoms.n_atoms != 8]

In [ ]:
path2out = '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/n2p2/Si/102/data'

with open(f'{path2out}/input.data', mode='a') as f:
    for structure_name, atoms_list in all_atoms_dict.items():
        for atoms in atoms_list:
            try:
                structure_id = atoms.structure_id if atoms.structure_id is not None else structure_name
                f.write('\n'.join(write_from_atoms(atoms, format='n2p2', structure_id=structure_id)))
            except:
                continue

# For prediction

In [4]:
from glob import glob
from mlptools.atoms.atom import MLPAtoms
import numpy as np

path2atoms = '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/atoms_near_injection'
all_atoms = glob(f'{path2atoms}/*.pkl')
print('number of atoms', len(all_atoms))

def ase_atom_to_mlpatom(path2atom, ase_atom):
    n_atoms = ase_atom.get_global_number_of_atoms()
    cell = np.array([vec for vec in ase_atom.cell])
    coord = ase_atom.get_positions()
    force = np.zeros((n_atoms, 3))
    energy = 0.0
    total_magnetization = 0.0
    structure_id = 'substrate'
    symbols = f'Si{n_atoms}'
    return MLPAtoms(
        cell=cell,
        coord=coord,
        force=force,
        energy=energy,
        n_atoms=n_atoms,
        total_magnetization=total_magnetization,
        structure_id=structure_id,
        symbols=symbols,
        path=path2atom
    )

number of atoms 100


In [3]:
path2out = '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/atoms_near_injection/n2p2_input'
path2model = '/Users/y1u0d2/desktop/Lab/result/lammps/models/Si/18'
all_mlpatoms = []
for atoms in all_atoms:
    #load atoms from pkl
    with open(atoms, mode='rb') as f:
        ase_atom = pickle.load(f)

    ase_atom.set_chemical_symbols(f'Si{ase_atom.get_global_number_of_atoms()}')
    mlpatom = ase_atom_to_mlpatom(path2atom=atoms, ase_atom=ase_atom)

    frame_id = mlpatom.path.split('/')[-1].split('.')[0]
    path2target = os.path.join(path2out, f'{frame_id}')
    os.makedirs(path2target, exist_ok=True)
    # copy input.nn scaling.data weights.014.data from path2model to path2target
    shutil.copy(os.path.join(path2model, 'input.nn'), path2target)
    shutil.copy(os.path.join(path2model, 'scaling.data'), path2target)
    shutil.copy(os.path.join(path2model, 'weights.014.data'), path2target)
    # write input.data
    with open(os.path.join(path2target, 'input.data'), mode='w') as f:
        f.write('\n'.join(write_from_atoms(mlpatom, format='n2p2')))

number of mlpatoms 0


## LAMMPSのinputファイルを作成する

In [80]:
path2template = '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/atoms_near_injection/lmp_input/template'
path2out = '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/atoms_near_injection/lmp_input'

def flatten_list(nested_list):
    flattened_list = []
    for item in nested_list:
        if isinstance(item, list):
            flattened_list.extend(flatten_list(item))
        else:
            flattened_list.append(item)
    return flattened_list


all_mlpatoms = []
for atoms in all_atoms:
    #load atoms from pkl
    with open(atoms, mode='rb') as f:
        ase_atom = pickle.load(f)

    ase_atom.set_chemical_symbols(f'Si{ase_atom.get_global_number_of_atoms()}')
    mlpatom = ase_atom_to_mlpatom(path2atom=atoms, ase_atom=ase_atom)

    with open(os.path.join(path2template, 'lmp.in'), mode='r') as f:
        lines = [s.strip() for s in f.readlines()]

    scaled_positions = ase_atom.get_scaled_positions()
    cell = np.array([vec for vec in ase_atom.cell])

    cell_for_lmp = []
    for i, vec in enumerate(cell):
        vec = list(map(lambda x: str(x), vec))
        cell_for_lmp.append([f'a{i+1}'] + vec + ['&'])

    scaled_positions_for_lmp = []
    for scaled_position in scaled_positions:
        scaled_position_str = []
        for x in scaled_position:
            if x > 1.0:
                x = 0.999999
            scaled_position_str.append(str('{:.6f}'.format(x)))  
        # scaled_position = list(map(lambda x: str('{:.6f}'.format(x)) , scaled_position))
        scaled_positions_for_lmp.append(['basis'] + scaled_position_str + ['&'])

    cell_for_lmp = [' '.join(cell) for cell in cell_for_lmp]
    scaled_positions_for_lmp = [' '.join(scaled_position) for scaled_position in scaled_positions_for_lmp]
    for i, l in enumerate(lines):
        if 'a1 ' in l:
            lines[i:i+3] = cell_for_lmp
            lines[i+3] = scaled_positions_for_lmp

    lines = flatten_list(lines)

    frame_id = mlpatom.path.split('/')[-1].split('.')[0]
    path2target = os.path.join(path2out, f'{frame_id}')
    if os.path.exists(path2target):
        raise Exception(f'{path2target} already exists')
    else:
        os.makedirs(path2target)
    
    with open(os.path.join(path2target, 'lmp.in'), mode='w') as f:
        # write lines
        f.write('\n'.join(lines))

In [74]:
cell_for_lmp

['a1 40.724999999999994 0.0 0.0 &',
 'a2 0.0 40.724999999999994 0.0 &',
 'a3 0.0 0.0 153.41921902474164 &']

In [76]:
lines

['atom_style      atomic',
 'units           metal',
 'boundary        p p p',
 'atom_modify map array sort 0 0',
 '',
 'lattice custom 1.0 &',
 'a1 40.724999999999994 0.0 0.0 &',
 'a2 0.0 40.724999999999994 0.0 &',
 'a3 0.0 0.0 153.41921902474164 &',
 ['basis 0.000133 0.000133 0.000035 &',
  'basis 0.000133 0.066800 0.017732 &',
  'basis 0.066800 0.000133 0.017732 &',
  'basis 0.066800 0.066800 0.000035 &',
  'basis 0.033467 0.033467 0.008884 &',
  'basis 0.033467 0.100133 0.026580 &',
  'basis 0.100133 0.033467 0.026580 &',
  'basis 0.100133 0.100133 0.008884 &',
  'basis 0.133467 0.000133 0.000035 &',
  'basis 0.133467 0.066800 0.017732 &',
  'basis 0.200133 0.000133 0.017732 &',
  'basis 0.200133 0.066800 0.000035 &',
  'basis 0.166800 0.033467 0.008884 &',
  'basis 0.166800 0.100133 0.026580 &',
  'basis 0.233467 0.033467 0.026580 &',
  'basis 0.233467 0.100133 0.008884 &',
  'basis 0.000133 0.133467 0.000035 &',
  'basis 0.000133 0.200133 0.017732 &',
  'basis 0.066800 0.133467 0

In [71]:
def flatten_list(nested_list):
    flattened_list = []
    for item in nested_list:
        if isinstance(item, list):
            flattened_list.extend(flatten_list(item))
        else:
            flattened_list.append(item)
    return flattened_list

['atom_style      atomic',
 'units           metal',
 'boundary        p p p',
 'atom_modify map array sort 0 0',
 '',
 'lattice custom 1.0 &',
 'a1 40.724999999999994 0.0 0.0 &',
 'a2 0.0 40.724999999999994 0.0 &',
 'a3 0.0 0.0 153.41921902474164 &',
 'basis 0.000133 0.000133 0.000035 &',
 'basis 0.000133 0.066800 0.017732 &',
 'basis 0.066800 0.000133 0.017732 &',
 'basis 0.066800 0.066800 0.000035 &',
 'basis 0.033467 0.033467 0.008884 &',
 'basis 0.033467 0.100133 0.026580 &',
 'basis 0.100133 0.033467 0.026580 &',
 'basis 0.100133 0.100133 0.008884 &',
 'basis 0.133467 0.000133 0.000035 &',
 'basis 0.133467 0.066800 0.017732 &',
 'basis 0.200133 0.000133 0.017732 &',
 'basis 0.200133 0.066800 0.000035 &',
 'basis 0.166800 0.033467 0.008884 &',
 'basis 0.166800 0.100133 0.026580 &',
 'basis 0.233467 0.033467 0.026580 &',
 'basis 0.233467 0.100133 0.008884 &',
 'basis 0.000133 0.133467 0.000035 &',
 'basis 0.000133 0.200133 0.017732 &',
 'basis 0.066800 0.133467 0.017732 &',
 'basis